In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c, e, m_p
from scipy.signal import fftconvolve

%matplotlib tk
import seaborn as sns
sns.set_context('notebook', font_scale=1.5,
                rc={'lines.markeredgewidth': 1})
sns.set_style('darkgrid', {
        'axes.linewidth': 2,
        'legend.fancybox': True})

/usr/lib64/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Bunch creation

We create a bunch. No fancy wrapper function available yet to produce a RF matched distribution. A little tedious because it requires quite some definitions to be done beforehand of the full RF System - but that's due to the nature of the problem.

- Some beam kinetic parameters and RF machine parameters

In [139]:
from PyHEADTAIL.trackers.simple_long_tracking import RFSystems
from PyHEADTAIL.trackers.rf_bucket import RFBucket
from PyHEADTAIL.particles.generators import ParticleGenerator, StationaryExponential
from PyHEADTAIL.particles.generators import gaussian2D, longitudinal_linear_matcher, RF_bucket_distribution

### SPS

In [140]:
# Machine
C      = 6911.
R      = C/(2*np.pi)

p0     = 26e9 * e/c
gamma  = np.sqrt(1 + (p0/(m_p*c))**2)

alpha  = 1/18**2
eta    = alpha - 1/gamma**2
V      = [4e6, -1e6]
h      = [4620, 18480]
dphi   = [np.pi, 0]

# Acceleration
beta          = np.sqrt(1-1/gamma**2)
T0            = C/(beta*c)
normalisation = 1/C * e/p0 * T0
dp            = 0*3.98e6 * e/c
# dp            = -1e6 * e/c

### LHC

In [79]:
momentum = 450e9 * e/c
gamma = np.sqrt((momentum/(m_p*c))**2 + 1)
beta = np.sqrt(1 - gamma**-2)

circumference = 26658.883
alpha = 3.225e-4
h1 = 35640
V1 = 8e6
p_increment = 0 * e/c * circumference/(beta*c)

print gamma

479.606062093


In [127]:
rfbucket = RFBucket(
    charge=e, mass=m_p, gamma=gamma,
    circumference=C,
    alpha_array=[alpha], p_increment=dp,
    harmonic_list=h, voltage_list=V, phi_offset_list=dphi)

In [128]:
eps_geo_z = 0.1 * e / (4. * np.pi * p0)

bunch = ParticleGenerator(
    macroparticlenumber=1e5, intensity=1e11,
    charge=e, mass=m_p, gamma=gamma,
    circumference=C,
    distribution_x=gaussian2D(2e-6),
    distribution_y=gaussian2D(2e-6),
    distribution_z=gaussian2D(eps_geo_z), Qs=0.01, eta=eta).generate()
#     distribution_z=RF_bucket_distribution(rfbucket, sigma_z=0.23)).generate()

In [138]:
fig = plt.figure(figsize=(16,10))
ax1 = fig.add_subplot(111)
hh = rfbucket.hamiltonian(bunch.z, bunch.dp)
zz = np.linspace(-.8, .8, 1000)
ss = rfbucket.separatrix(zz)

ax1.plot(zz, +ss, c='orange', lw=2)
ax1.plot(zz, -ss, c='orange', lw=2)
ax1.scatter(bunch.z, bunch.dp, c=hh, marker='.', cmap=plt.cm.viridis_r)
ax1.set_xlim(-.9, .9)
ax1.set_ylim(-5e-3, 5e-3)
plt.xlabel("z [m]")
plt.ylabel(r"$\delta$")
plt.show()

In [135]:
long_map = RFSystems(
    circumference=C,
    harmonic_list=h, voltage_list=V, phi_offset_list=dphi, alpha_array=[alpha],
    gamma_reference=gamma, p_increment=dp, mass=m_p, charge=e)

In [136]:
plt.close('all')
plt.ion()
fig, ax1 = plt.subplots(1, figsize=(16, 10))

nturns = 100
turns = np.arange(nturns)
mean_z = 0.*turns
mean_dp = 0.*turns
sigma_z = 0.*turns
epsn_z = 0.*turns
for i in xrange(nturns):
#     print '{:d} of {:d}\r'.format(i, nturns)
    long_map.track(bunch)
    mean_z[i] = bunch.mean_z()
    mean_dp[i] = bunch.mean_dp()
    sigma_z[i] = bunch.sigma_z()
    epsn_z[i] = bunch.epsn_z()
    if i%2==0:
        ax1.plot(zz, +ss, c='orange', lw=2)
        ax1.plot(zz, -ss, c='orange', lw=2)
        ax1.plot(bunch.z[::10], bunch.dp[::10], '.')
        plt.pause(0.1)
        plt.draw()
        plt.cla()

plt.ioff()
plt.close('all')

In [132]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(16,10), sharex=True)

ax1.plot(turns, mean_z)
ax1.set_ylabel(r"$\mu_z$ [m]")
ax1.set_ylim(-1, 1)
ax2.plot(turns, sigma_z)
ax2.set_xlabel("z [m]")
ax2.set_ylabel(r"$\sigma_z$ [eV s]")
ax2.set_ylim(0, 0.5)

plt.show()

## Wakes, slicing, convolution etc.

The CircularResonator wake is a special resonator wake with Yokoya factors X1=1, Y1=1, X2=0, Y2=0

functions transverse and longitudinal are actually meant to be used internally when building the wake kicks - at this point they are concatenated with the respective Yokoya factor being applied/provided. For visualisation we can also build them externally manually.

In [19]:
from PyHEADTAIL.impedances.wakes import CircularResonator, \
    WakeField, check_wake_sampling
from PyHEADTAIL.particles.slicing import UniformBinSlicer

In [29]:
wake = CircularResonator(R_shunt=1e9, frequency=1e9, Q=20)
slicer = UniformBinSlicer(60, z_cuts=(-.3, .3))

wakefields = WakeField(slicer, wake)
kick = wakefields.wake_kicks[0]
wf = kick.wake_function

slices = []
times, cmoments = [], []
for i, b in enumerate(bunches):
    distance = dt[i]*b.beta*c
    b.z += distance
    slices.append(b.get_slices(slicer))
    times.append(slices[-1].z_centers / (b.beta*c))
    cmoments.append(slices[-1].charge_per_slice)
    b.z -= distance

slices1, slices2, slices3 = slices
times1, times2, times3 = times
cmoments1, cmoments2, cmoments3 = cmoments

*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!
*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!


In [21]:
col = sns.hls_palette(6, l=.3, s=.8)

fig, axes = plt.subplots(len(slices), figsize=(12,6))

for i, ax in enumerate(axes):
    [ax.axvline(s, c=col[0]) for s in slices[i].z_centers]
    [ax.axvline(s, c=col[2]) for s in [slices[i].z_cut_tail, slices[i].z_cut_head]]
    ax.plot(slices[i].z_centers, slices[i].charge_per_slice, '-o')
    ax.set_xlim(-.4, .4)

plt.show()

## Convolutions

In [82]:
def convolution_python(t_target, t_source, c_source, w):
    dxp = 0.*t_target
    for k in xrange(len(t_target)):
        for l in xrange(len(t_source)):
            dxp[k] += c_source[l]*w(t_target[k]-t_source[l])

    return dxp

In [88]:
def convolution_numpy(t_target, t_source, c_source, w):
    tmin, tmax = t_source[0], t_source[-1]
    tt = np.concatenate((t_target-tmax, (t_target-tmin)[1:]))
    
    return np.convolve(c_source, w(tt), mode='valid')

In [84]:
def convolution_scipy(t, x, w):
    tmin, tmax = t[0], t[-1]
    tt = np.concatenate((t-tmax, (t-tmin)[1:]))
    
    return fftconvolve(w(tt), x, mode='valid')

In [91]:
def convolution_multibunch_numpy(times, moments, wf, dt):
    
    dxp = []
    if dt is None: dt = 0.*np.array(times)
    for i in xrange(len(times)):
        z = 0.*times[i]
        t_target = times[i]
        for j in range(i+1):
            t_source = times[j] + dt[i] - dt[j]
            c_source = moments[j]
            z += convolution_numpy(t_target, t_source, c_source, wf)
            
        dxp.append(z)
        
    return dxp

In [ ]:
def convolution_multibunch_numpy(times, cmoments, wf, dt):
    dts = times[i]
    t0 = np.concatenate((dts-dts[-1], (dts-dts[0])[1:]))

    dxp = np.convolve(s.charge_per_slice, wf(t0))
    dxp_s = np.convolve(s.charge_per_slice, wf(t0), mode='same')
    dxp_v = np.convolve(s.charge_per_slice, wf(t0), mode='valid')

    for j in range(i):
        print i, j, dt[i-j]
        t_sources = times[j] + dt[i]-dt[j]
        t2 = np.concatenate((dts-t_sources[-1], (dts-t_sources[0])[1:]))

        dxp += np.convolve(slices[j].charge_per_slice, wf(t2))
        dxp_s += np.convolve(slices[j].charge_per_slice, wf(t2), mode='same')
        dxp_v += np.convolve(slices[j].charge_per_slice, wf(t2), mode='valid')
    
    dxp_list.append(dxp_v)

In [85]:
def convolve_multibunch(times, moments, wf, dt=None):
    dxp = []
    if dt is None: dt = 0.*np.array(times)
    for i in xrange(len(times)):
        z = 0.*times[i]
        t_target = times[i]
        for j in range(i+1):
            t_source = times[j] + dt[i] - dt[j]
            c_source = moments[j]
            z += convolution_python(t_target, t_source, c_source, wf)
            
        dxp.append(z)
        
    return dxp

In [ ]:
plt.close('all')